In [ ]:
%pip install pandas numpy matplotlib seaborn plotly scipy statsmodels

In [27]:
#@title Environment Setup

from google.cloud import bigquery

import os
import pandas as pd
import numpy as np
import datetime
import warnings
warnings.filterwarnings('ignore')
from typing import List

# plot
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.cbook import boxplot_stats
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# regression analysis
import scipy.stats as stats
import statsmodels.api as sm


pd.set_option('display.max_columns', None)


client = bigquery.Client(project='moloco-ods')
def process_query(input_query):
    job_config = bigquery.QueryJobConfig()
    query_job = client.query(input_query, job_config=job_config)
    df_return = query_job.result().to_dataframe()
    return df_return

def process_query_be(input_query):
    job_config = bigquery.QueryJobConfig()
    query_job = client.query(input_query, job_config=job_config)
    print(f"Submitted job: {query_job.job_id}")
    return query_job

def fetch_result(query_job):
    """
    Return DataFrame only if the query is complete and successful.
    If still running, return None.
    If failed, return a dict with error info (so next loop can still check other jobs).
    """
    try:
        if query_job.done():
            if query_job.error_result:
                # Query finished but failed
                print(f"Job {query_job.job_id} failed: {query_job.error_result}")
                return {"status": "error", "job_id": query_job.job_id, "error": query_job.error_result}
            else:
                # Query finished successfully
                print(f"Job {query_job.job_id} is complete!")
                df_return = query_job.result().to_dataframe()
                return {"status": "success", "job_id": query_job.job_id, "data": df_return}
        else:
            # Query still running
            print(f"Job {query_job.job_id} is still running...")
            return None
    except Exception as e:
        # Unexpected error while fetching results
        print(f"Job {query_job.job_id} raised an exception: {e}")
        return {"status": "exception", "job_id": query_job.job_id, "error": str(e)}


In [28]:
#@title data pull

## start date and end date, target quarters (2 consecutive qurters)
start_date = '2025-07-01'
end_date = '2025-12-31'
tgt_quarters = ['Q3','Q4']

query = f"""

  WITH market AS (
    SELECT
        (FORMAT_DATE('%Y-%m', DATE_TRUNC(fact_market.install_date_utc , QUARTER))) AS date_utc_quarter,
        fact_market.migrated_platform  AS platform_id,
        SAFE_DIVIDE(COALESCE(SUM(fact_market.moloco.revenue_d7 ), 0), COALESCE(SUM(fact_market.moloco.revenue_d7 ), 0) + COALESCE(SUM(fact_market.non_moloco.revenue_d7 ), 0)) AS share_of_revenue_d7
    FROM `ads-bpd-guard-china.market_share.fact_market`  AS fact_market
    WHERE ((( fact_market.install_date_utc  ) >= (DATE('{start_date}')) AND ( fact_market.install_date_utc  ) <= (DATE('{end_date}'))))
      AND (fact_market.office ) = 'KOR'
    GROUP BY
        1,
        2
  ),

  spend AS (
    SELECT
        (FORMAT_TIMESTAMP('%Y-%m', TIMESTAMP_TRUNC(TIMESTAMP(fact_dsp_core.date_utc) , QUARTER))) AS date_utc_quarter,
        fact_dsp_core.platform_id  AS platform_id,
        COALESCE(SUM(fact_dsp_core.gross_spend_usd ), 0) AS spend
    FROM `ads-bpd-guard-china.athena.fact_dsp_core` fact_dsp_core
    WHERE ((( TIMESTAMP(fact_dsp_core.date_utc)  ) >= (TIMESTAMP('{start_date} 00:00:00')) AND ( TIMESTAMP(fact_dsp_core.date_utc)  ) < (TIMESTAMP('{end_date} 00:00:00'))))
      AND (fact_dsp_core.advertiser.office ) = 'KOR'
    GROUP BY
        1,
        2
      HAVING spend > 0
  )

  SELECT
    *
  FROM spend LEFT JOIN market USING(platform_id, date_utc_quarter)
  ORDER BY platform_id, date_utc_quarter

"""

In [29]:
df = process_query(query)
df.head()

ValueError: Please install the 'pandas' package to use this function.